In [24]:
import pandas as pd
import numpy as np
import jalali
import statistics

# Modify Date And Water Table
# Date Type: 
#   1.Gregorian (Text): 2000-11-1
#   2.Persian (Text): 1399-1-1
# Date & Value:
#   Column of Pandas Dataframe
def convert_to_day_15(info, date, value, date_type="persian"):
    df = info.copy()
    if date_type == "gregorian":
        df["DATE_GREGORIAN"] = date.apply(pd.to_datetime)
        df["DATE_PERSIAN"] = list(
            map(
                lambda i: jalali.Gregorian(i.date()).persian_string(),
                df["DATE_GREGORIAN"]
            )
        )
    elif date_type == "persian":
        df["DATE_PERSIAN"] = date
        df["DATE_GREGORIAN"] = list(
            map(
                lambda i: jalali.Persian(i).gregorian_string(),
                df["DATE_PERSIAN"]
            )
        )
        df["DATE_GREGORIAN"] = df["DATE_GREGORIAN"].apply(pd.to_datetime)    
    else:
        pass
    df["VALUE"] = value        
    df["DELTA_DAY"] = df["DATE_GREGORIAN"].diff().dt.days
    df["DATE_PERSIAN_NEW"] = list(
        map(
            lambda i: f"{int(i.split('-')[0])}-{int(i.split('-')[1])}-{15}",
            df["DATE_PERSIAN"]
        )
    )
    df["DATE_GREGORIAN_NEW"] = list(
        map(
            lambda i: jalali.Persian(i).gregorian_string(),
            df["DATE_PERSIAN_NEW"]
        )
    )
    df["DATE_GREGORIAN_NEW"] = df["DATE_GREGORIAN_NEW"].apply(pd.to_datetime)
    df["VALUE_NEW"] = df["VALUE"]
    A = []
    A.append(df["VALUE"][0])
    for i in range(1, len(df)):
        if int(df["DATE_PERSIAN"][i].split('-')[2]) >= 15:
            NEW_VALUE = df["VALUE"][i-1] + ((((df["DATE_GREGORIAN_NEW"][i] - df["DATE_GREGORIAN"][i-1]).days) / ((df["DATE_GREGORIAN"][i] - df["DATE_GREGORIAN"][i-1]).days)) * (df["VALUE"][i] - df["VALUE"][i-1]))
            A.append(NEW_VALUE)
        else:
            NEW_VALUE = df["VALUE"][i] + ((((df["DATE_GREGORIAN_NEW"][i] - df["DATE_GREGORIAN"][i]).days) / ((df["DATE_GREGORIAN"][i+1] - df["DATE_GREGORIAN"][i]).days)) * (df["VALUE"][i+1] - df["VALUE"][i]))
            A.append(NEW_VALUE)
    df["VALUE_NEW"] = A
    return df


# Read Data And GeoInfo
xls = pd.ExcelFile('Data/CSV/HydrographData.xlsx')
Data = pd.read_excel(xls, sheet_name='Data')
GeoInfo = pd.read_excel(xls, sheet_name='GeoInfo')

# GeoInfo
COLs = ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'LOCATION_NAME']
GeoInfo[COLs] = GeoInfo[COLs].apply(lambda x: x.str.rstrip())
GeoInfo[COLs] = GeoInfo[COLs].apply(lambda x: x.str.lstrip())

# Data
COLs = ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'LOCATION_NAME']
Data[COLs] = Data[COLs].apply(lambda x: x.str.rstrip())
Data[COLs] = Data[COLs].apply(lambda x: x.str.lstrip())

# Convert Date
Data["DATE_GREGORIAN_RAW"] = Data["DATE_GREGORIAN_RAW"].apply(pd.to_datetime)

Data['DATE_CHECK'] = np.where(
    Data["DATE_PERSIAN_RAW"].isna(),
    np.where(
        Data["DATE_GREGORIAN_RAW"].isna(),
        np.NaN,
        "G"
    ),
    "P"  
)

Data['DATE_PERSIAN_RAW'] = Data.apply(
    lambda x: jalali.Gregorian(x["DATE_GREGORIAN_RAW"].date()).persian_string() if x["DATE_CHECK"] == "G" else x["DATE_PERSIAN_RAW"], 
    axis=1
)

Data['DATE_GREGORIAN_RAW'] = Data.apply(
    lambda x: jalali.Persian(x["DATE_PERSIAN_RAW"]).gregorian_string() if x["DATE_CHECK"] == "P" else x["DATE_GREGORIAN_RAW"], 
    axis=1
)

Data["DATE_GREGORIAN_RAW"] = Data["DATE_GREGORIAN_RAW"].apply(pd.to_datetime)


Data.drop(['DATE_CHECK'], axis=1, inplace=True)

Data.sort_values(
    by=["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", "DATE_GREGORIAN_RAW"], 
    inplace=True
)

# Remove NanN Data In Column "WATER_TABLE_RAW"
Data.dropna(
    subset=["WATER_TABLE_RAW"],
    inplace=True
)

Data.reset_index(
    inplace=True,
    drop=True
)

wt_date_converted = convert_to_day_15(
    info=Data[["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME"]],
    date=Data["DATE_PERSIAN_RAW"],
    value=Data["WATER_TABLE_RAW"],
    date_type="persian"
)[["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", "DATE_PERSIAN", "DATE_PERSIAN_NEW", "DATE_GREGORIAN", "DATE_GREGORIAN_NEW", "VALUE_NEW"]]

wt_date_converted.columns = ["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", "DATE_PERSIAN_RAW", "DATE_PERSIAN", "DATE_GREGORIAN_RAW","DATE_GREGORIAN", "WATER_TABLE"]

Data = Data.merge(
    right=wt_date_converted,
    how="left",
    on=["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", "DATE_PERSIAN_RAW", "DATE_GREGORIAN_RAW"]
)

# Data['TMP'] = (Data['STORAGE_COEFFICIENT_LOCATION'] * Data['THISSEN_LOCATION']) / Data['THISSEN_AQUIFER']

# tmp = Data.groupby(by=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN']).sum().reset_index()[
#             ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN', 'TMP']].rename(columns={'TMP': 'STORAGE_COEFFICIENT_AQUIFER'})

# Data = Data.merge(
#     right=tmp,
#     how='left',
#     on=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN'])\
#         .sort_values(["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", 'DATE_GREGORIAN'])\
#             .drop(['TMP'], axis=1)

# del tmp

# SELECTED_LEVEL = 'LEVEL_SRTM'

# Data = Data.merge(
#     right=GeoInfo[['MAHDOUDE_NAME', 'AQUIFER_NAME', 'LOCATION_NAME', SELECTED_LEVEL]],
#     how='left',
#     on=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'LOCATION_NAME']
# ).rename(columns={SELECTED_LEVEL: 'LEVEL'})

# Data['HEAD_LOCATION'] = Data['LEVEL'] - Data['WATER_TABLE']

# Data['TMP'] = (Data['HEAD_LOCATION'] * Data['THISSEN_LOCATION']) / Data['THISSEN_AQUIFER']

# tmp = Data.groupby(by=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN']).sum().reset_index()[
#             ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN', 'TMP']].rename(columns={'TMP': 'HEAD_AQUIFER_THISSEN'})

# Data = Data.merge(
#     right=tmp,
#     how='left',
#     on=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'DATE_PERSIAN'])\
#         .sort_values(["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", 'DATE_GREGORIAN'])\
#             .drop(['TMP'], axis=1)

# del tmp

# tmp = Data[['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'HEAD_LOCATION']]\
#     .groupby(by=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN'])\
#         .agg({'HEAD_LOCATION': [statistics.mean, statistics.geometric_mean, statistics.harmonic_mean]})\
#             .reset_index()

# tmp.columns = [col for col in tmp.columns]

# tmp.columns = ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN', 'HEAD_AQUIFER_ARITHMETIC', 'HEAD_AQUIFER_GEOMETRIC', 'HEAD_AQUIFER_HARMONIC']

# Data = Data.merge(
#     right=tmp,
#     how='left',
#     on=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'DATE_GREGORIAN'])\
#         .sort_values(["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", 'DATE_GREGORIAN'])

# del tmp


Data

,MAHDOUDE_NAME,MAHDOUDE_CODE,AQUIFER_NAME,LOCATION_NAME,LOCATION_STATE,DATE_PERSIAN_RAW,DATE_GREGORIAN_RAW,WATER_TABLE_RAW,NO_MEASURE_CODE,INFO,STORAGE_COEFFICIENT_LOCATION,THISSEN_LOCATION,THISSEN_AQUIFER,DATE_PERSIAN,DATE_GREGORIAN,WATER_TABLE
0,سرخس,6004,سرخس,s.6,O,1361-1-25,1982-04-14,3.91,NaN,NaN,0.05,NaN,NaN,1361-1-15,1982-04-04,3.910000
1,سرخس,6004,سرخس,s.6,O,1361-2-18,1982-05-08,3.76,NaN,NaN,0.05,NaN,NaN,1361-2-15,1982-05-05,3.778750
2,سرخس,6004,سرخس,s.6,O,1361-4-28,1982-07-19,4.95,NaN,NaN,0.05,NaN,NaN,1361-4-15,1982-07-06,4.735139
3,سرخس,6004,سرخس,s.6,O,1361-5-26,1982-08-17,5.29,NaN,NaN,0.05,NaN,NaN,1361-5-15,1982-08-06,5.161034
4,سرخس,6004,سرخس,s.6,O,1361-6-25,1982-09-16,5.44,NaN,NaN,0.05,NaN,NaN,1361-6-15,1982-09-06,5.390000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10517,سرخس,6004,سرخس,یاس تپه راه جنگل,O,1396-4-26,2017-07-17,21.84,NaN,NaN,0.05,NaN,NaN,1396-4-15,2017-07-06,21.821143
10518,سرخس,6004,سرخس,یاس تپه راه جنگل,O,1396-5-12,2017-08-03,21.98,NaN,NaN,0.05,NaN,NaN,1396-5-15,2017-08-06,22.004667
10519,سرخس,6004,سرخس,یاس تپه راه جنگل,O,1396-6-26,2017-09-17,22.35,NaN,NaN,0.05,NaN,NaN,1396-6-15,2017-09-06,22.259556
10520,سرخس,6004,سرخس,یاس تپه راه جنگل,O,1396-7-27,2017-10-19,22.34,NaN,NaN,0.05,NaN,NaN,1396-7-15,2017-10-07,22.343750


In [119]:

def create_date_day15(min, max):
    
    result = []

    min_list = list(map(lambda x: int(x), min.split("-")))
    max_list = list(map(lambda x: int(x), max.split("-")))

    for y in range(min_list[0], max_list[0] + 1):
        for m in range(1, 13):
            result.append(f"{y}-{m}-15")

    result = pd.DataFrame(
        {"DATE_PERSIAN" : result}
    )

    result['DATE_GREGORIAN'] = result.apply(
        lambda x: jalali.Persian(x["DATE_PERSIAN"]).gregorian_string(), 
        axis=1
    )

    result["DATE_GREGORIAN"] = result["DATE_GREGORIAN"].apply(pd.to_datetime)

    result = result[result["DATE_GREGORIAN"] >= pd.to_datetime(jalali.Persian(min).gregorian_string())]
    result = result[result["DATE_GREGORIAN"] <= pd.to_datetime(jalali.Persian(max).gregorian_string())]    
    
    return result



for mn in list(Data["MAHDOUDE_NAME"].unique()):
    for an in list(Data["AQUIFER_NAME"].unique()):
        for ln in list(Data["LOCATION_NAME"].unique()):

            df = Data[(Data["MAHDOUDE_NAME"] == mn) & (Data["MAHDOUDE_NAME"] == an) & (Data["LOCATION_NAME"] == ln)]

            date = create_date_day15(
                min = df.DATE_PERSIAN.min(),
                max = df.DATE_PERSIAN.max()
            )

            df = date.merge(
                df,
                how="left",
                on=["DATE_PERSIAN", "DATE_GREGORIAN"]
            )

            df[]

            df.to_excel("dd.xlsx")


            break
        break
    break





In [14]:
import pandas as pd
import numpy as np

df = pd.DataFrame(
    [
        np.nan,
        4.95,
        5.29,
        5.44,
        5.49,
        5.31,
        5.08,
        4.99,
        5,
        np.nan,
        4.27,
        3.39,
        3.97,
        4.45,
        np.nan,
        np.nan,
        np.nan,
        np.nan,
        5.15,
        4.7,
        4.71,
        4.71,
        4.57,
        np.nan,
        5.12,
        5.62,
        np.nan
    ],
    columns=['a']
)

df_result = df.interpolate(method='from_derivatives')



print(pd.DataFrame({
    "RAW":df.a,
    "ddd":df_result.a
})) 


     RAW    ddd
0    NaN    NaN
1   4.95  4.950
2   5.29  5.290
3   5.44  5.440
4   5.49  5.490
5   5.31  5.310
6   5.08  5.080
7   4.99  4.990
8   5.00  5.000
9    NaN  4.635
10  4.27  4.270
11  3.39  3.390
12  3.97  3.970
13  4.45  4.450
14   NaN  4.590
15   NaN  4.730
16   NaN  4.870
17   NaN  5.010
18  5.15  5.150
19  4.70  4.700
20  4.71  4.710
21  4.71  4.710
22  4.57  4.570
23   NaN  4.845
24  5.12  5.120
25  5.62  5.620
26   NaN    NaN
